<h1>Table des matières<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Traitement-du-df-aircraft" data-toc-modified-id="Traitement-du-df-aircraft-1">Traitement du df aircraft</a></span></li><li><span><a href="#Traitement-du-df-airline" data-toc-modified-id="Traitement-du-df-airline-2">Traitement du df airline</a></span></li><li><span><a href="#Traitement-du-df-airport" data-toc-modified-id="Traitement-du-df-airport-3">Traitement du df airport</a></span></li><li><span><a href="#Traitement-du-df-country" data-toc-modified-id="Traitement-du-df-country-4">Traitement du df country</a></span></li><li><span><a href="#Traitement-du-df-city" data-toc-modified-id="Traitement-du-df-city-5">Traitement du df city</a></span></li><li><span><a href="#Traitement-du-df-routes" data-toc-modified-id="Traitement-du-df-routes-6">Traitement du df routes</a></span></li><li><span><a href="#Faire-correspondre-colonnes-des-df-avec-les-champs-des-tables-sql" data-toc-modified-id="Faire-correspondre-colonnes-des-df-avec-les-champs-des-tables-sql-7">Faire correspondre colonnes des df avec les champs des tables sql</a></span></li></ul></div>

In [1]:
import pandas as pd
import numpy as np
import os
import json
import requests
from os import listdir
from os.path import join, isfile, isdir
from datetime import datetime, timedelta
from urllib.parse import unquote

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
%%html
<style>
.dataframe thead tr th {
    text-align: left;
}
.dataframe thead th {
    text-align: left;
}
</style>

In [3]:
path_folder_data_api = "static_results"
path_folder_data_wiki = "data_scraping"
path_folder_output_csv = "csv_to_sql"

In [4]:
# on convertit nos données en DataFrame et on affiche le nom des colonnes

files = [f for f in listdir(path_folder_data_api) if isfile(join(path_folder_data_api, f))]
dic_columns = {}
all_df = []
for file in files:
    path_file = f"{path_folder_data_api}/{file}"
    with open(path_file, "r", encoding='utf-8') as f:
        items = json.load(f)
                
    data = pd.json_normalize(items)
    df = pd.DataFrame(data)
    
    all_df.append(df)
    cols = list(df.columns)
    dic_columns[file[:-5]]=cols
    
display(dic_columns)

{'aircraft': ['AircraftCode', 'AirlineEquipCode', 'NameOK'],
 'airlines': ['AirlineID', 'AirlineID_ICAO', 'NameOK'],
 'airports': ['AirportCode',
  'CityCode',
  'CountryCode',
  'LocationType',
  'UtcOffset',
  'TimeZoneId',
  'NameOK',
  'Position.Coordinate.Latitude',
  'Position.Coordinate.Longitude'],
 'cities': ['CityCode',
  'CountryCode',
  'UtcOffset',
  'TimeZoneId',
  'NameOK',
  'Airports.AirportCode'],
 'countries': ['CountryCode', 'NameOK']}

In [5]:
# on crée un dictionnaire pour renommer les colonnes

dic_columns_new = {
    'aircraft': [
        'aircraft_iata',
        'aircraft_icao',
        'aircraft_name'
    ],
    'airlines': [
        'airline_iata',
        'airline_icao',
        'airline_name'
    ],
    'airports': [
        'airport_iata',
        'airport_city_code',
        'airport_country_code',
        'airport_location_type',
        'airport_utc_offset',
        'airport_timezone_id',
        'airport_name',
        'airport_latitude',
        'airport_longitude'
    ],
    'cities': [
        'city_code',
        'city_country_code',
        'city_utc_offset',
        'city_timezone_id',
        'city_name',
        'city_airport_code'
    ],
    'countries': [
        'country_code', 
        'country_name'
    ]
}

In [6]:
# on nomme les df et on utilise dic_columns_new pour changer le nom des colonnes

aircraft = all_df[0]
airline = all_df[1]
airport = all_df[2]
city = all_df[3]
country = all_df[4]

df_data = [
    aircraft,
    airline,
    airport,
    city,
    country
]

for i in range(len(df_data)):
    df_data[i].columns = dic_columns_new[files[i][:-5]]

In [7]:
# création d'un dictionnaire de df avec les NaN
def get_nan_from_df(df):
    cols = df.loc[:, df.isna().any()].columns
    dic_result = {}
    dic_result['df_nan'] = df[df.isna().any(axis = 1)]
    dic_result['col_nan'] = []
    for col in cols:
        indices_nan = df[[col]][df[[col]].isna().any(axis = 1)].index
        dic = {
            "col_name": col,
            "nb_nan": len(df.loc[df[col].isna()]),
            "df": df[df.index.isin(indices_nan)]
        }
        dic_result['col_nan'].append(dic)
    return dic_result

# Affichage des NaN
def display_nan(df):
    df_nan = get_nan_from_df(df)
    print("Synthèse :")
    dic_summary = {dic['col_name']: dic['nb_nan'] for dic in df_nan['col_nan']}
    display(dic_summary)
    #display(df_nan['df_nan'])
    for dic in df_nan['col_nan']:
        print(f'Nb de NaN dans la colonne "{dic["col_name"]}" : {dic["nb_nan"]}')
        display(dic['df'])

## Traitement du df aircraft

In [8]:
# affichage du df
print(aircraft.shape)
display(aircraft.sample(10))

(382, 3)


,aircraft_iata,aircraft_icao,aircraft_name
42,70M,B703,Boeing 707-320B / 320C Mixed Configuration
261,EC5,EC55,Eurocopter EC155
7,221,BCS1,Airbus A220-100
285,GJ5,GLF5,Gulfstream Aerospace V (G500/G550)
116,789,B789,Boeing 787-9
369,T2F,T204,Tupolev Tu-204 Freighter
271,F23,F28,Fokker F28 Fellowship 3000
114,783,B783,Boeing 787-3
275,F5F,F50,Fokker 50 Freighter
17,31N,A19N,Airbus A139neo


In [9]:
# Verification des NaN
display_nan(aircraft)

Synthèse :


{'aircraft_icao': 2, 'aircraft_name': 2}

Nb de NaN dans la colonne "aircraft_icao" : 2


,aircraft_iata,aircraft_icao,aircraft_name
77,748,NaN,None
78,748,NaN,None


Nb de NaN dans la colonne "aircraft_name" : 2


,aircraft_iata,aircraft_icao,aircraft_name
77,748,NaN,None
78,748,NaN,None


In [10]:
# Lignes sans données => on les supprime
aircraft = aircraft.dropna()
aircraft = aircraft.reset_index(drop=True)

In [11]:
# on regarde si la colonne aircraft_iata ne comporte que des valeurs uniques
iata = aircraft['aircraft_iata'].tolist()
iata_uniques = list(set(iata))
print(len(iata))
print(len(iata_uniques))

380
379


In [12]:
# on affiche les lignes avec le même code_iata
for iata in iata_uniques:
    df = aircraft[aircraft['aircraft_iata'] == iata]
    if len(df) > 1:
        display(df)

,aircraft_iata,aircraft_icao,aircraft_name
97,762,B762,Boeing 767-200 Passenger
98,762,B762,Boeing 767-200 Passenger


In [13]:
# on supprime les rows dupliquées
aircraft = aircraft.drop_duplicates()
aircraft = aircraft.reset_index(drop=True)
iata = aircraft['aircraft_iata'].tolist()
iata_uniques = list(set(iata))
print(len(iata))
print(len(iata_uniques))

379
379


In [14]:
# on va ajouter les données scrappées sur wikipedia pour ajouter un lien vers la page wiki

# on vérifie les données du scraping
with open(f"{path_folder_data_wiki}/aircrafts_wiki.json", 'r', encoding='utf-8') as f:
    aircrafts_wiki = json.load(f)
aircraft_wiki = pd.DataFrame(aircrafts_wiki)
aircraft_wiki

,ICAO,IATA,MODEL,Link
0,A124,A4F,Antonov An-124 Ruslan,/wiki/Antonov_An-124_Ruslan
1,A140,A40,Antonov An-140,/wiki/Antonov_An-140
2,A148,A81,Antonov An-148,/wiki/Antonov_An-148
3,A158,A58,Antonov An-158,/wiki/Antonov_An-148#Variants
4,A19N,31N,Airbus A319neo,/wiki/Airbus_A319neo
...,...,...,...,...
325,WW24,WWP,Israel Aircraft Industries 1124 Westwind,/wiki/IAI_Westwind
326,Y12,YN2,Harbin Y-12,/wiki/Harbin_Y-12
327,YK40,YK4,Yakovlev Yak-40,/wiki/Yakovlev_Yak-40
328,YK42,YK2,Yakovlev Yak-42,/wiki/Yakovlev_Yak-42


In [15]:
# on vérifie la présence de valeurs uniques pour le code iata
aircraft_wiki = aircraft_wiki.dropna(subset=["IATA"])
aircraft_wiki = aircraft_wiki[aircraft_wiki['IATA'].isin(iata_uniques)]
iata_wiki = aircraft_wiki['IATA'].tolist()
iata_wiki_uniques = list(set(iata_wiki))
print(len(iata_wiki))
print(len(iata_wiki_uniques))

253
233


In [16]:
# on traite les valeurs multiples du code iata :
# pour les codes multtiples, on recherche la correspondance du code icao avec notre df aircraft
# si pas de correpondance, on supprime la row du df wiki
index_del = []
for iata in iata_wiki_uniques:
    df = aircraft_wiki[aircraft_wiki['IATA'] == iata]
    if len(df) > 1:
        df_index = list(df.index)
        df_air = aircraft[aircraft['aircraft_iata'] == iata]
        code_icao = df_air['aircraft_icao'].tolist()[0]
        if code_icao != "*":
            for i in df_index:
                if df.loc[i, 'ICAO'] != code_icao:
                    index_del.append(i)
        else:
            index_del.extend(df_index)
aircraft_wiki = aircraft_wiki.drop(index_del, axis=0)

In [17]:
# on merge les 2 df
aircraft_wiki.columns = ['icao', 'aircraft_iata', 'model', 'aircraft_link']
aircraft_wiki = aircraft_wiki[['aircraft_iata', 'aircraft_link']]
aircraft = pd.merge(aircraft, aircraft_wiki, how='left')
display(aircraft)

,aircraft_iata,aircraft_icao,aircraft_name,aircraft_link
0,100,F100,Fokker 100,/wiki/Fokker_100
1,141,B461,BAE Systems 146-100 Passenger,/wiki/British_Aerospace_146
2,142,B462,BAE Systems 146-200 Passenger,/wiki/British_Aerospace_146
3,143,B463,BAE Systems 146-300 Passenger,/wiki/British_Aerospace_146
4,14X,B461,BAE Systems 146-100 Freighter,NaN
...,...,...,...,...
374,YK2,YK42,Yakovlev Yak-42 / Yak-142,/wiki/Yakovlev_Yak-42
375,YK4,YK40,Yakovlev Yak-40,/wiki/Yakovlev_Yak-40
376,YN2,Y12,Harbin Yunshuji Y12,/wiki/Harbin_Y-12
377,YN7,AN24,Xian Yunshuji Y7,NaN


## Traitement du df airline

In [18]:
# affichage du df
airline = all_df[1]
print(airline.shape)
display(airline.sample(10))

(1136, 3)


,airline_iata,airline_icao,airline_name
326,CW,CCC,Air Cargo Global
1128,ZV,NaN,V Air Corporation
247,AT,RAM,Royal Air Maroc
1016,W7,WLB,Wings Of Lebanon
570,JJ,TAM,Latam Airlines Brazil
309,CE,CLG,Chalair
858,RU,ABW,Airbridgecargo
800,Q7,SBM,Sky Bahamas
554,J4,BDR,Badr Airlines
186,8P,PCO,Pacific Coastal Airlines


In [19]:
# Vérification des nan
display_nan(airline)

Synthèse :


{'airline_icao': 125}

Nb de NaN dans la colonne "airline_icao" : 125


,airline_iata,airline_icao,airline_name
14,1X,NaN,Branson Airexpress
15,2A,NaN,Deutsche Bahn AG
17,2C,NaN,Sncf
19,2E,NaN,Smokey Bay Air
28,2O,NaN,Island Air Service
...,...,...,...
1128,ZV,NaN,V Air Corporation
1132,ZZ,NaN,Alc
1133,ZZ,NaN,Alc
1134,ZZ,NaN,Alc


In [20]:
# On supprime les 3 dernières lignes 
airline = airline.drop([1133, 1134, 1135], axis=0)

# on regarde si la colonne airline_iata ne comporte que des valeurs uniques
print(airline.shape)
uniques_iata = airline['airline_iata'].unique().tolist()
print(len(uniques_iata))

(1133, 3)
1127


In [21]:
# On vérifie que tous les codes iata ne comportent que 2 caractères
list_id = []
for i in range(len(airline)):
    c = airline.loc[i, 'airline_iata']
    if len(c) != 2:
        list_id.append(i)

print(len(airline[airline.index.isin(list_id)]))
display(airline[airline.index.isin(list_id)].sample(5))

41


,airline_iata,airline_icao,airline_name
775,PGT,PGT,Pegasus Hava Tasimaciligi A.S. (Pegasus)
436,FUA,FUA,Futura International
352,DNV,DNV,Aeroflot-Don_ Joint-Stock Company
950,UCR,UCR,Aero-Charter Ukraine Ltd
269,BCS,BCS,European Air Transport


In [22]:
# On va utiliser les données scrappées sur wikipedia pour modifier les données
with open(f"{path_folder_data_wiki}/airlines_wiki.json", "r", encoding='utf-8') as f:
    airlines_wiki = json.load(f)
print(airlines_wiki[:1])

airline_icao_to_iata = {}
for entry in airlines_wiki:
    airline_icao = entry['ICAO']
    iata_code = entry['IATA']
    if len(iata_code) == 2:
        airline_icao_to_iata[airline_icao] = iata_code

all_iata_code = airline['airline_iata'].unique().tolist()
id_to_del = []
for idx, row in airline.iterrows():
    if idx in list_id:
        airline_icao = row['airline_icao']
        if airline_icao in airline_icao_to_iata:
            iata_code = airline_icao_to_iata[airline_icao]
            if iata_code not in all_iata_code:
                all_iata_code.append(iata_code)
                airline.at[idx, 'airline_iata'] = airline_icao_to_iata[airline_icao]
            else:
                id_to_del.append(idx)
        else:
            id_to_del.append(idx)
            
if len(id_to_del) != 0:
    airline = airline.drop(id_to_del, axis=0)
    airline = airline.reset_index(drop = True)
    airline = airline.drop_duplicates()
    airline = airline.reset_index(drop = True)
    
print(airline.shape)

[{'IATA': 'PR', 'ICAO': 'BOI', 'Airline': '2GO', 'Link': '/wiki/2GO_(cargo_airline)', 'CallSign': 'ABAIR', 'Country': 'Philippines', 'Comment': ''}]
(1091, 3)


In [23]:
# on regarde les rows avec un code iata multiple
count = pd.Series(airline['airline_iata'].tolist()).value_counts()
codes_iata = []
for i in range(len(count)):
    if count[i]>1:
        codes_iata.append(count.index[i])
        
display(airline[airline['airline_iata'].isin(codes_iata)])

,airline_iata,airline_icao,airline_name
88,4Y,BGA,Airbus Transport International
89,4Y,OCN,Eurowings Discover
606,LH,DLH,Lufthansa
607,LH,GEC,Lufthansa Cargo


In [24]:
# On supprime les lignes dupliquées
airline = airline.drop([89, 607], axis=0)
airline = airline.reset_index(drop = True)
print(airline.shape)

(1089, 3)


In [25]:
airline.dtypes

airline_iata    object
airline_icao    object
airline_name    object
dtype: object

In [26]:
# Airline : On traite les codes ICAO multiples
duplicated = airline.duplicated(subset='airline_icao', keep=False)
airline_duplicated = airline[duplicated]
temp = airline_duplicated.sort_values('airline_icao')
temp['airline_icao'] = temp['airline_icao'].astype(str)
temp = temp[temp['airline_icao'] != 'nan']
temp

,airline_iata,airline_icao,airline_name
480,HT,AHW,Aeromist-Kharkiv Ltd
835,S8,AHW,Sky Capital Airlines
183,8R,EDW,Edelweiss Air Ag
984,WK,EDW,Edelweiss Air
364,EF,FEA,Far Eastern Air Transport Corporation
397,FE,FEA,Far Eastern Air Transport
660,N3,OMS,Omskavia Airline_ Public Joint-Stock Society
719,OV,OMS,Salamair
413,FV,SDM,Rossiya Airlines
796,R4,SDM,State Transport Company _russia_


In [27]:
drop_index = [835, 183, 364, 397, 660, 796, 6, 33]
airline = airline.drop(drop_index, axis=0)
airline = airline.reset_index(drop=True)

## Traitement du df airport

In [28]:
# affichage du df
print(airport.shape)
display(airport.sample(10))

(11681, 9)


,airport_iata,airport_city_code,airport_country_code,airport_location_type,airport_utc_offset,airport_timezone_id,airport_name,airport_latitude,airport_longitude
8215,SJB,SJB,BO,Airport,-04:00,America/La_Paz,San Joaquin,-13.0833,-64.8000
1589,CPE,CPE,MX,Airport,-06:00,America/Mexico_City,Campeche A.Acuna Ongay Intl,19.8167,-90.5003
9375,USN,USN,KR,Airport,+09:00,Asia/Seoul,Ulsan,35.5936,129.3517
1507,CMA,CMA,AU,Airport,+10:00,Australia/Brisbane,Cunnamulla,-28.0300,145.6222
1485,CLE,CLE,US,Airport,-05:00,America/New_York,Cleveland,41.4117,-81.8497
8367,SOY,SOY,GB,Airport,+00:00,Europe/London,Stronsay,59.1553,-2.6419
11297,ZII,ZII,AU,Harbour,+09:30,Australia/Darwin,Adelaide Ferry Port,-34.8167,138.5167
1093,BUX,BUX,CD,Airport,+02:00,Africa/Lubumbashi,Bunia,1.5658,30.2208
11007,YVC,YVC,CA,Airport,-06:00,America/Regina,La Ronge Barber Field,55.0833,-105.3333
10811,YLT,YLT,CA,Airport,-05:00,America/Toronto,Alert,82.5167,-62.2833


In [29]:
# location_type : on se limite aux airports
airport = airport[airport['airport_location_type'] == "Airport"]

# on supprime la colonne 'airport_location_type'
airport = airport.drop(columns=['airport_location_type'])

airport = airport.reset_index(drop=True)
airport.shape

(9728, 8)

In [30]:
# on regarde si la colonne airport_iata ne comporte que des valeurs uniques
iata_code = airport["airport_iata"].tolist()
print(len(iata_code))
iata_code_uniques = (list(set(iata_code)))
print(len(iata_code_uniques))

9728
9723


In [31]:
# on regarde les rows avec un code iata multiple
count = pd.Series(airport['airport_iata'].tolist()).value_counts()
codes_iata = []
for i in range(len(count)):
    if count[i]>1:
        codes_iata.append(count.index[i])
        
display(airport[airport['airport_iata'].isin(codes_iata)])

,airport_iata,airport_city_code,airport_country_code,airport_utc_offset,airport_timezone_id,airport_name,airport_latitude,airport_longitude
97,ADX,ADX,GB,+00:00,Europe/London,St Andrews Raf Leuchars,56.3667,-2.8667
98,ADX,ADX,GB,+00:00,Europe/London,St Andrews Raf Leuchars,56.3667,-2.8667
2840,GOM,GOM,CD,+02:00,Africa/Lubumbashi,None,-1.6708,29.2383
2841,GOM,GOM,CD,+02:00,Africa/Lubumbashi,Goma,-1.6708,29.2383
2861,GPS,GPS,EC,-06:00,Pacific/Galapagos,None,-0.4539,-90.2658
2862,GPS,GPS,EC,-06:00,Pacific/Galapagos,Baltra Island Seymour,-0.4539,-90.2658
3046,HBX,HBX,IN,+05:30,Asia/Kolkata,Hubli,15.3617,75.0869
3047,HBX,HBX,IN,+05:30,Asia/Kolkata,None,15.3617,75.0869
3068,HDN,HDN,US,-07:00,America/Denver,Hayden Yampa Valley,40.4833,-107.2236
3069,HDN,HDN,US,-07:00,America/Denver,None,40.4833,-107.2236


In [32]:
# on supprime les rows dupliquées
airport = airport.drop([98, 2840, 2861, 3047, 3069], axis=0)
airport = airport.reset_index(drop = True)
print(airport.shape)

(9723, 8)


In [33]:
display_nan(airport)

Synthèse :


{}

In [34]:
# on utilise les données airport scrappées sur wikipedia pour ajouter le code ICAO et un lien

# on vérifie les données
with open(f"{path_folder_data_wiki}/airports_wiki.json", "r", encoding='utf-8') as f:
    airports_wiki = json.load(f)
airports_wiki[:3]

for dic in airports_wiki:
    if '[' in dic['ICAO']:
        dic['ICAO'] = dic['ICAO'][:4]
    if isinstance(dic['AirportLink'], str):
        if '"' in dic['AirportLink']:
            dic['AirportLink'] = dic['AirportLink'].replace('"', '')

In [35]:
# on renomme les colonnes pour faire correspondre avec notre df airport
df_wiki = pd.DataFrame(airports_wiki)
df_wiki = df_wiki[['IATA', 'ICAO', 'AirportLink']]
df_wiki.columns = ['airport_iata', 'airport_icao', 'airport_link']
df_wiki

,airport_iata,airport_icao,airport_link
0,AAA,NTGA,/wiki/Anaa_Airport
1,AAB,YARY,/wiki/Arrabury_Airport
2,AAC,HEAR,/wiki/El_Arish_International_Airport
3,AAD,HCAD,/wiki/Adado_Airport
4,AAE,DABB,/wiki/Rabah_Bitat_Airport
...,...,...,...
9075,ZYL,VGSY,/wiki/Osmani_International_Airport
9076,ZZE,UBBZ,/wiki/Zangilan_International_Airport
9077,ZZO,UHSO,/wiki/Zonalnoye_Airport
9078,ZZU,FWUU,/wiki/Mzuzu_Airport


In [36]:
# on merge les 2 df
airport = pd.merge(airport, df_wiki, how='left')
airport

,airport_iata,airport_city_code,airport_country_code,airport_utc_offset,airport_timezone_id,airport_name,airport_latitude,airport_longitude,airport_icao,airport_link
0,AAA,AAA,PF,-10:00,Pacific/Tahiti,Anaa,-17.3525,-145.5100,NTGA,/wiki/Anaa_Airport
1,AAB,AAB,AU,+10:00,Australia/Brisbane,Arrabury,-26.6911,141.0472,YARY,/wiki/Arrabury_Airport
2,AAC,AAC,EG,+02:00,Africa/Cairo,El Arish International,31.0733,33.8358,HEAR,/wiki/El_Arish_International_Airport
3,AAD,AAD,SO,+03:00,Africa/Mogadishu,Adado,6.0961,46.6375,HCAD,/wiki/Adado_Airport
4,AAE,AAE,DZ,+01:00,Africa/Algiers,Annaba Rabah Bitat,36.8222,7.8092,DABB,/wiki/Rabah_Bitat_Airport
...,...,...,...,...,...,...,...,...,...,...
9718,ZZE,ZZE,AZ,+04:00,Asia/Baku,Zangilan,39.0989,46.7344,UBBZ,/wiki/Zangilan_International_Airport
9719,ZZO,ZZO,RU,+11:00,Asia/Srednekolymsk,Zonalnoye,50.6689,142.7611,UHSO,/wiki/Zonalnoye_Airport
9720,ZZU,ZZU,MW,+02:00,Africa/Blantyre,Mzuzu,-11.4447,34.0117,FWUU,/wiki/Mzuzu_Airport
9721,ZZV,ZZV,US,-05:00,America/New_York,Zanesville Municipal,39.9500,-81.8833,KZZV,/wiki/Zanesville_Municipal_Airport


In [37]:
# Airport : On traite les codes ICAO multiples
duplicated = airport.duplicated(subset='airport_icao', keep=False)
airport_duplicated = airport[duplicated]
airport_duplicated = airport_duplicated[['airport_iata', 'airport_icao']]\
                        .astype(str)\
                        .replace(['nan', ''], [np.nan, np.nan])\
                        .dropna()
airport_duplicated.sort_values('airport_icao')

,airport_iata,airport_icao
997,BSL,LFSB
5210,MLH,LFSB
7106,RZS,OPSW
7677,SWN,OPSW
1629,CTP,SNCP
2141,EEA,SNCP
2818,GNJ,UBBG
4363,KVD,UBBG
5784,NMT,VYNT
5943,NYT,VYNT


In [38]:
drop_index = [5210, 7677, 1629, 4363, 5784, 7546]
airport = airport.drop(drop_index, axis=0)

In [39]:
airport = airport.reset_index(drop=True)

## Traitement du df country

In [40]:
# affichage du df
print(country.shape)
display(country.sample(10))

(239, 2)


,country_code,country_name
196,SO,Somalia
111,KN,"Saint Kitts And Nevis, Leeward Island"
26,BN,Brunei Darussalam
89,HK,Hong Kong
225,VE,Venezuela
14,AZ,Azerbaijan
232,XK,Kosovo Republic Of
97,IN,India
74,GD,"Grenada, Windward Islands"
36,CC,Cocos (Keeling) Islands


In [41]:
# Affichage des NaN
display_nan(country)

Synthèse :


{}

In [42]:
# on verifie que code_pays sont uniques
codes_pays = country['country_code']
print(len(codes_pays))
codes_pays_uniques = list(set(codes_pays))
print(len(codes_pays_uniques))

239
238


In [43]:
# on regarde les rows avec un code pays multiple
count = pd.Series(country['country_code'].tolist()).value_counts()
codes_pays_ = []
for i in range(len(count)):
    if count[i]>1:
        codes_pays_.append(count.index[i])
        
display(country[country['country_code'].isin(codes_pays_)])

,country_code,country_name
99,IR,Iran Islamic Republic Of
100,IR,Iran Islamic Republic Of


In [44]:
# on supprime les rows dupliquées
country = country.drop([100], axis=0)
country = country.reset_index(drop = True)
print(country.shape)

(238, 2)


In [45]:
# on verifie que tous les codes country de airport sont présents
codes_pays_airports = airport['airport_country_code'].unique().tolist()
missing = list(set(codes_pays_airports) - set(codes_pays))
print(missing)

[]


In [46]:
# on utilise les données scrappées sur wikipedia pour rajouter des infos

# on vérifie les données
with open(f"{path_folder_data_wiki}/country_wiki.json", "r", encoding='utf-8') as f:
    country_wiki = json.load(f)
df_wiki = pd.DataFrame(country_wiki)
df_wiki.head()

,CountryName,CountryLink,CountryFlag,Alpha2Code,Alpha3Code,NumericCode
0,Afghanistan,/wiki/Afghanistan,//upload.wikimedia.org/wikipedia/commons/thumb...,AF,AFG,004
1,Åland Islands,/wiki/%C3%85land_Islands,//upload.wikimedia.org/wikipedia/commons/thumb...,AX,ALA,248
2,Albania,/wiki/Albania,//upload.wikimedia.org/wikipedia/commons/thumb...,AL,ALB,008
3,Algeria,/wiki/Algeria,//upload.wikimedia.org/wikipedia/commons/thumb...,DZ,DZA,012
4,American Samoa,/wiki/American_Samoa,//upload.wikimedia.org/wikipedia/commons/thumb...,AS,ASM,016


In [47]:
# on renomme les colonnes pour faire correspondre avec notre df country
df_wiki = df_wiki[['Alpha2Code', 'Alpha3Code', 'NumericCode', 'CountryFlag', 'CountryLink']]
df_wiki.columns = ['country_code', 'country_iso3', 'country_num', 'country_flag', 'country_link']

In [48]:
# on merge les 2 df
country = pd.merge(country, df_wiki, how='left')
country

,country_code,country_name,country_iso3,country_num,country_flag,country_link
0,AD,Andorra,AND,020,//upload.wikimedia.org/wikipedia/commons/thumb...,/wiki/Andorra
1,AE,United Arab Emirates,ARE,784,//upload.wikimedia.org/wikipedia/commons/thumb...,/wiki/United_Arab_Emirates
2,AF,Afghanistan,AFG,004,//upload.wikimedia.org/wikipedia/commons/thumb...,/wiki/Afghanistan
3,AG,"Antigua And Barbuda, Leeward Islands",ATG,028,//upload.wikimedia.org/wikipedia/commons/thumb...,/wiki/Antigua_and_Barbuda
4,AI,"Anguilla, Leeward Islands",AIA,660,//upload.wikimedia.org/wikipedia/commons/thumb...,/wiki/Anguilla
...,...,...,...,...,...,...
233,YE,Yemen,YEM,887,//upload.wikimedia.org/wikipedia/commons/thumb...,/wiki/Yemen
234,YT,Mayotte,MYT,175,//upload.wikimedia.org/wikipedia/en/thumb/c/c3...,/wiki/Mayotte
235,ZA,South Africa,ZAF,710,//upload.wikimedia.org/wikipedia/commons/thumb...,/wiki/South_Africa
236,ZM,Zambia,ZMB,894,//upload.wikimedia.org/wikipedia/commons/thumb...,/wiki/Zambia


## Traitement du df city

In [49]:
# affichage du df
print(city.shape)
display(city.sample(10))

(10700, 6)


,city_code,city_country_code,city_utc_offset,city_timezone_id,city_name,city_airport_code
5022,MLW,LR,+00:00,Africa/Monrovia,Monrovia,"[MLW, ROB]"
5143,MQS,VC,-04:00,America/St_Vincent,Mustique,MQS
679,BGE,US,-05:00,America/New_York,Bainbridge,BGE
3382,IOW,US,-06:00,America/Chicago,Iowa City,IOW
5681,NVR,RU,+03:00,Europe/Moscow,Novgorod,NVR
1464,COA,US,-08:00,America/Los_Angeles,Columbia,COA
3078,HOI,PF,-10:00,Pacific/Tahiti,Hao,HOI
4064,KPL,PG,+10:00,Pacific/Port_Moresby,Kapal,KPL
375,ARV,US,-06:00,America/Chicago,Minocqua/Woodruff,ARV
4342,LDW,AU,+08:00,Australia/Perth,Landsdowne Station,LDW


In [50]:
# Affichage des NaN
display_nan(city)

Synthèse :


{'city_airport_code': 104}

Nb de NaN dans la colonne "city_airport_code" : 104


,city_code,city_country_code,city_utc_offset,city_timezone_id,city_name,city_airport_code
21,AAW,PK,+05:00,Asia/Karachi,Abbottabad,NaN
66,ACQ,ES,+01:00,Europe/Madrid,Estacion De Alicante,NaN
191,AIY,US,-05:00,America/New_York,Atlantic City,NaN
279,ANA,KE,+03:00,Africa/Nairobi,Masai Mara,NaN
433,AUB,BR,-04:00,America/Cuiaba,Itauba,NaN
...,...,...,...,...,...,...
9396,XGL,FR,+01:00,Europe/Paris,Angouleme,NaN
9562,XPI,FR,+01:00,Europe/Paris,Gare Perpignan Sncf,NaN
9652,XTZ,FI,+02:00,Europe/Helsinki,Turku,NaN
9697,XWA,US,-06:00,America/Chicago,Williston,NaN


In [51]:
# on verifie que code_city sont uniques
codes_city = city['city_code']
print(len(codes_city))
codes_city_uniques = list(set(codes_city))
print(len(codes_city_uniques))

10700
10693


In [52]:
# on regarde les rows avec un code pays multiple
count = pd.Series(city['city_code'].tolist()).value_counts()
codes_city_ = []
for i in range(len(count)):
    if count[i]>1:
        codes_city_.append(count.index[i])
        
display(city[city['city_code'].isin(codes_city_)])

,city_code,city_country_code,city_utc_offset,city_timezone_id,city_name,city_airport_code
99,AEB,CN,+08:00,Asia/Shanghai,Baise,AEB
100,AEB,CN,+08:00,Asia/Shanghai,Baise,AEB
2536,GBI,IN,+05:30,Asia/Kolkata,Gulbarga,GBI
2537,GBI,IN,+05:30,Asia/Kolkata,Gulbarga,GBI
3491,IXX,IN,+05:30,Asia/Kolkata,Bidar Airport,IXX
3492,IXX,IN,+05:30,Asia/Kolkata,Bidar,IXX
3493,IXX,IN,+05:30,Asia/Kolkata,Bidar,IXX
9026,WAS,US,-05:00,America/New_York,Washington,"[BOF, BWI, BZS, DCA, GBO, IAD, JPN, MTN, ZBP, ..."
9027,WAS,US,-05:00,America/New_York,Washington,"[BOF, BWI, BZS, DCA, GBO, IAD, JPN, MTN, ZBP, ..."
9028,WAS,US,-05:00,America/New_York,Washington,"[BOF, BWI, BZS, DCA, GBO, IAD, JPN, MTN, ZBP, ..."


In [53]:
# on supprime les rows dupliquées
city = city.drop([100, 2537, 3492, 3493, 9027, 9028, 10223], axis=0)
city = city.reset_index(drop = True)
print(city.shape)

(10693, 6)


In [54]:
# on verifie que tous les codes city de airport sont présents ; si non, on supprime de airports les lignes où le code est manquant
codes_city_airports = airport['airport_city_code'].unique().tolist()
missing = list(set(codes_city_airports) - set(codes_city))

if len(missing) != 0:
    missing_id = list(airport[airport['airport_iata'].isin(missing)].index)
    airport = airport.drop(missing_id, axis=0)
    airport = airport.reset_index(drop=True)
    
    # verif que tous les codes sont présents
    new_codes_city_airports = airport['airport_city_code'].unique().tolist()
    test_missing = list(set(new_codes_city_airports) - set(codes_city))
    if len(test_missing) != 0:
        print("Erreur : Tous les codes city du df airport ne sont pas présents dans le df city")

In [55]:
# on supprime la colonne city_airport_code de city 
# en effet :
#   - soit la ville possède un aéroport et alors city_airport_code = city_code
#   - soit la ville possède plusieurs aéroports et alors la liste des city_airport_code correspond aux airport_iata du df airport
#
# Exemple avec le city_code 'WAS' du df city:
display(city[city['city_code'] == 'WAS'])

# on contsate que cette ville possède plusieurs aéroports car la valeur de la variable city_airport_code est une liste de code iata d'aéroport
# on peut vérifier avec le 1ere code de cette liste, 'BOF', qu'il s'agit bien d'un airport_code et que la ville associée est bien 'WAS'
display(airport[airport['airport_iata'] == 'BOF'])

# on va donc supprimer la colonne city_airport_code du df city
# (de plus, il est possible que tous les codes présents dans les listes ne soient pas présents dans le df airport, alors autant supprimer cette colonne)

city = city.drop(columns=['city_airport_code'])

,city_code,city_country_code,city_utc_offset,city_timezone_id,city_name,city_airport_code
9022,WAS,US,-05:00,America/New_York,Washington,"[BOF, BWI, BZS, DCA, GBO, IAD, JPN, MTN, ZBP, ..."


,airport_iata,airport_city_code,airport_country_code,airport_utc_offset,airport_timezone_id,airport_name,airport_latitude,airport_longitude,airport_icao,airport_link
909,BOF,WAS,US,-05:00,America/New_York,Washington Bolling Afb Heliport,38.9167,-77.0167,NaN,NaN


In [56]:
city.sample(3)

,city_code,city_country_code,city_utc_offset,city_timezone_id,city_name
4168,KUP,PG,+10:00,Pacific/Port_Moresby,Kupiano
9713,XXF,NZ,+12:00,Pacific/Auckland,Scenic Flight
247,ALT,BR,-03:00,America/Araguaina,Alenquer


## Traitement du df routes

In [57]:
# Ajout des données des principales routes aériennes
routes = pd.read_csv("data_csv/routes.tsv", sep="\t", dtype="str")
len(routes)

translation = pd.read_csv("data_csv/translation.tsv", sep="\t", dtype="str")
translation = translation[['Operator', 'Operator_correct']]
translation.columns = ['CallSign', 'NewCallSign']
routes = pd.merge(routes, translation, how='left')
routes['CallSign'] = routes['NewCallSign'].combine_first(routes['CallSign'])
routes = routes.drop(columns=['NewCallSign'])
routes = routes.drop_duplicates(subset=['CallSign'], keep='last')
routes = routes.dropna(subset=['FromAirport_ICAO', 'ToAirport_ICAO'])
routes = routes.reset_index(drop=True)
routes

,CallSign,Operator_ICAO,FromAirport_ICAO,FromAirport_Time,ToAirport_ICAO,ToAirport_Time,RouteStop
0,AFR1328,NaN,LFPG,15:35,LIPE,17:15,NaN
1,VLG8027,NaN,LFPO,15:15,LEBL,16:50,NaN
2,HOP8216,NaN,LFMT,21:45,EDDK,23:15,NaN
3,RYR5223,NaN,LFOB,19:55,GMFF,21:50,NaN
4,BER7548,NaN,EDDK,04:30,LEPA,06:45,NaN
...,...,...,...,...,...,...,...
827179,FX1043,NaN,MSP,NaN,GSO,NaN,NaN
827180,FX1057,NaN,JAX,NaN,GSO,NaN,NaN
827181,OO3299,NaN,ORD,NaN,CLE,NaN,NaN
827182,5Y4531,NaN,ANC,NaN,CHS,NaN,NaN


In [58]:
# Nos données sont identifiées par le code iata
# Or dans le df routes, les données sont identifiées par le code icao
# On va donc convertir les codes icao de route en code iata

# on crée un diction des code icao et iata du df airport
airport_dict = airport.set_index('airport_icao')['airport_iata'].to_dict()

In [59]:
# Convertir les codes icao en code iata
def icao_to_iata(x):
    # Si x n'est pas un str, on retourne NaN
    if not isinstance(x, str):
        return np.nan

    # Si len(x) == 3, on retourne x
    if len(x) == 3:
        return x

    # Si len(x) == 4
    if len(x) == 4:
        # Si x est une clé de airport_dic, on retourne airport_dic[x], sinon NaN
        return airport_dict.get(x, np.nan)

    # Si len(x) != 3 and len(x) != 4, on retourne NaN
    return np.nan

cols = ['FromAirport_ICAO', 'ToAirport_ICAO']
for col in cols:
    routes[col] = routes[col].apply(icao_to_iata)

routes = routes.dropna(subset=['FromAirport_ICAO', 'ToAirport_ICAO'])
routes = routes.reset_index(drop=True)
routes = routes.rename(columns={
    'FromAirport_ICAO': 'FromAirport_IATA',
    'ToAirport_ICAO': 'ToAirport_IATA'
})
routes

,CallSign,Operator_ICAO,FromAirport_IATA,FromAirport_Time,ToAirport_IATA,ToAirport_Time,RouteStop
0,AFR1328,NaN,CDG,15:35,BLQ,17:15,NaN
1,VLG8027,NaN,ORY,15:15,BCN,16:50,NaN
2,HOP8216,NaN,MPL,21:45,CGN,23:15,NaN
3,RYR5223,NaN,BVA,19:55,FEZ,21:50,NaN
4,BER7548,NaN,CGN,04:30,PMI,06:45,NaN
...,...,...,...,...,...,...,...
793954,FX1043,NaN,MSP,NaN,GSO,NaN,NaN
793955,FX1057,NaN,JAX,NaN,GSO,NaN,NaN
793956,OO3299,NaN,ORD,NaN,CLE,NaN,NaN
793957,5Y4531,NaN,ANC,NaN,CHS,NaN,NaN


In [60]:
# On supprime les routes dont les codes iata airport ne sont pas identifiés dans le df airport
iata_routes = routes['FromAirport_IATA'].unique().tolist()
iata_routes.extend(routes['ToAirport_IATA'].unique().tolist())
iata_routes = list(set(iata_routes))

iata_airports = airport['airport_iata'].unique().tolist()

missing = list(set(iata_routes) - set(iata_airports))

if len(missing) != 0:
    not_missing = list(set(iata_routes) - set(missing))
    routes = routes.loc[routes['FromAirport_IATA'].isin(not_missing)]
    routes = routes.loc[routes['ToAirport_IATA'].isin(not_missing)]
    routes = routes.reset_index(drop=True)

In [61]:
# On ajoute une colonne Operator_IATA et on supprime Operator_ICAO
air_temp = airline.dropna(subset=['airline_icao'])
airline_dict = air_temp.set_index('airline_icao')['airline_iata'].to_dict()
routes['Operator_IATA'] = routes['Operator_ICAO'].apply(lambda x: airline_dict.get(x, np.nan))
routes = routes.drop(columns=['Operator_ICAO'])

In [62]:
## On va créer un df pour les escales (on fera une table escales)
escales = routes['RouteStop'].dropna().unique().tolist()
df_escales = routes[routes['RouteStop'].isin(escales)][['CallSign', 'RouteStop']]

# on crée un dictionnaire avec pour clé le callsign et pour valeur un array des escales
dic_escales_temp = df_escales.set_index('CallSign')['RouteStop'].to_dict()
dic_escales = {}
for k,v in dic_escales_temp.items():
    v_array = v.strip().split()
    v_array_ok = []
    test = True
    for code in v_array:
        try:
            v_array_ok.append(airport_dict[code])
        except:
            test = False
            break
    if test:
        dic_escales[k] = v_array_ok
dic_escales

{'NVR828': ['DXB'],
 'GLG38': ['GYE'],
 'OAW349': ['BDS'],
 'OAW355': ['PMO'],
 'OAW367': ['CTA'],
 'NTJ331': ['LPI'],
 'NTJ332': ['LPI'],
 'NTJ333': ['LPI'],
 'NTJ334': ['LPI'],
 'NTJ335': ['LPI'],
 'NTJ336': ['LPI'],
 'GIP5151': ['FRA'],
 'JSA721': ['TPE'],
 'JSA722': ['TPE'],
 'JSA763': ['MNL'],
 'BOX410': ['ORD'],
 'BOX513': ['DXB'],
 'BOX550': ['ICN'],
 'CSC8503': ['XUZ'],
 'CSC8681': ['CZX'],
 'CSC8682': ['CZX'],
 'CSC8711': ['YIH'],
 'CSC8724': ['HET'],
 'CSC8725': ['WUH'],
 'CSC8792': ['JIU'],
 'CSC8813': ['XUZ'],
 'CSC8814': ['XUZ'],
 'CSC8839': ['NKG'],
 'CSC8840': ['NKG'],
 'CSC8843': ['TNA'],
 'CSC8844': ['TNA'],
 'CSC8849': ['TYN'],
 'CSC8858': ['CGO'],
 'CSC8862': ['TSN'],
 'CSC8877': ['SJW'],
 'CSC8878': ['SJW'],
 'CSC8897': ['CKG'],
 'CSC8927': ['KHN'],
 'CSC8928': ['KHN'],
 'CSC8930': ['TNA'],
 'CSC8952': ['CSX'],
 'CSC8953': ['CGO'],
 'CSC8986': ['WUH'],
 'TAY1': ['SIN'],
 '5O5614': ['CDG'],
 'TAY12G': ['EMA'],
 'TAY132L': ['MMX'],
 'TAY134J': ['MMX'],
 'TAY16': ['PVG

In [63]:
# On créé le df escales
routes['RouteStop'] = False
data_escales  = []
for k,v in dic_escales.items():
    for i,escale in enumerate(v):
        dic = {}
        dic['escale_route_callsign'] = k
        dic['escale_number_total'] = len(v)
        dic['escale_order_number'] = i+1
        dic['airport_iata'] = escale
        data_escales.append(dic)

df_escales = pd.DataFrame(data_escales)
df_escales.sort_values(by=['escale_route_callsign', 'escale_order_number'])

,escale_route_callsign,escale_number_total,escale_order_number,airport_iata
25643,3V642,1,1,BER
44,5O5614,1,1,CDG
23197,A54012,1,1,PIS
22669,A5614,2,1,NTE
22670,A5614,2,2,SNR
...,...,...,...,...
24266,YZR7490,2,1,ANC
24267,YZR7490,2,2,PVG
24607,YZR7498,1,1,TSN
25476,YZR7542,1,1,HYN


In [64]:
# On modifie le df routes pour remplacer les valeurs de StopRoute par un booleen hasStopover
routes.loc[routes['CallSign'].isin(list(dic_escales.keys())), 'RouteStop'] = True

## Faire correspondre colonnes des df avec les champs des tables sql

In [65]:
# Pour chaque df, on va s'assurer que le formatage correspond à la db mysql :
#  - type des variables du df doit correspondre au type défini dans la db
#  - ordre des colonnes des df doit être identique à celui de la base sql

In [66]:
# Convertir utc_offset en minutes
def convert_utc_offset_to_minutes(offset_str):
    if offset_str.startswith('-'):
        sign = -1
        offset_str = offset_str[1:]
    else:
        sign = 1
        offset_str = offset_str[1:] if offset_str.startswith('+') else offset_str

    hours, minutes = map(int, offset_str.split(':'))
    return sign * (hours * 60 + minutes)

def decode_url(url):
    if isinstance(url, str):
        if '"' in url:
            url = url.replace('"', '')
        if ',' in url:
            url = ''
        return url
    return url

In [67]:
print(airport.columns)
print(airport.dtypes)

Index(['airport_iata', 'airport_city_code', 'airport_country_code',
       'airport_utc_offset', 'airport_timezone_id', 'airport_name',
       'airport_latitude', 'airport_longitude', 'airport_icao',
       'airport_link'],
      dtype='object')
airport_iata             object
airport_city_code        object
airport_country_code     object
airport_utc_offset       object
airport_timezone_id      object
airport_name             object
airport_latitude        float64
airport_longitude       float64
airport_icao             object
airport_link             object
dtype: object


In [68]:
airport['airport_utc_offset_min'] = airport['airport_utc_offset'].apply(convert_utc_offset_to_minutes)

airport = airport[['airport_iata', 'airport_icao', 'airport_city_code', 'airport_country_code',
                   'airport_name', 'airport_utc_offset', 'airport_utc_offset_min', 'airport_timezone_id', 
                   'airport_latitude', 'airport_longitude', 'airport_link']]

airport.columns = ['airport_iata', 'airport_icao', 'fk_city_iata', 'fk_country_iso2',
                   'airport_name', 'airport_utc_offset_str', 'airport_utc_offset_min', 'airport_timezone_id', 
                   'airport_latitude', 'airport_longitude', 'airport_link']

airport['airport_link'] = airport['airport_link'].apply(decode_url)

airport.dtypes

airport_iata               object
airport_icao               object
fk_city_iata               object
fk_country_iso2            object
airport_name               object
airport_utc_offset_str     object
airport_utc_offset_min      int64
airport_timezone_id        object
airport_latitude          float64
airport_longitude         float64
airport_link               object
dtype: object

In [69]:
print(airline.columns)
print(airline.dtypes)

Index(['airline_iata', 'airline_icao', 'airline_name'], dtype='object')
airline_iata    object
airline_icao    object
airline_name    object
dtype: object


In [70]:
print(aircraft.columns)
print(aircraft.dtypes)

Index(['aircraft_iata', 'aircraft_icao', 'aircraft_name', 'aircraft_link'], dtype='object')
aircraft_iata    object
aircraft_icao    object
aircraft_name    object
aircraft_link    object
dtype: object


In [71]:
aircraft['aircraft_link'] = aircraft['aircraft_link'].apply(decode_url)
aircraft.columns = ['aircraft_iata', 'aircraft_icao', 'aircraft_name', 'aircraft_wiki_link']

In [72]:
print(country.columns)
print(country.dtypes)

Index(['country_code', 'country_name', 'country_iso3', 'country_num',
       'country_flag', 'country_link'],
      dtype='object')
country_code    object
country_name    object
country_iso3    object
country_num     object
country_flag    object
country_link    object
dtype: object


In [73]:
country = country[['country_code', 'country_iso3', 'country_name', 'country_num',
                   'country_link','country_flag']]

country.columns = ['country_iso2', 'country_iso3', 'country_name', 'country_numeric', 'country_wiki_link', 'country_flag']

country['country_numeric'] = country['country_numeric'].fillna(-1).astype(int)
country['country_wiki_link'] = country['country_wiki_link'].apply(decode_url)
country['country_flag'] = country['country_flag'].apply(decode_url)

print(country.dtypes)

country_iso2         object
country_iso3         object
country_name         object
country_numeric       int32
country_wiki_link    object
country_flag         object
dtype: object


In [74]:
print(city.columns)
print(city.dtypes)

Index(['city_code', 'city_country_code', 'city_utc_offset', 'city_timezone_id',
       'city_name'],
      dtype='object')
city_code            object
city_country_code    object
city_utc_offset      object
city_timezone_id     object
city_name            object
dtype: object


In [75]:
city['city_utc_offset_min'] = city['city_utc_offset'].apply(convert_utc_offset_to_minutes)
city = city[['city_code', 'city_name', 'city_country_code', 'city_utc_offset', 'city_utc_offset_min', 'city_timezone_id']]
city.columns = ['city_iata', 'city_name', 'fk_country_iso2', 'city_utc_offset_str', 'city_utc_offset_min', 'city_timezone_id']

In [76]:
print(routes.columns)
print(routes.dtypes)

Index(['CallSign', 'FromAirport_IATA', 'FromAirport_Time', 'ToAirport_IATA',
       'ToAirport_Time', 'RouteStop', 'Operator_IATA'],
      dtype='object')
CallSign            object
FromAirport_IATA    object
FromAirport_Time    object
ToAirport_IATA      object
ToAirport_Time      object
RouteStop             bool
Operator_IATA       object
dtype: object


In [77]:
def traitement_time(x):
    if isinstance(x, str):
        if len(x) == 5 and x[2] == ":":
            test = True
            p1 = x[:2]
            p2 = x[3:]
            if int(p1) < 0 or int(p1) > 23:
                test = False
            if int(p2) < 0 or int(p2) > 59:
                test = False
            if test:
                d = datetime.strptime(x, "%H:%M")
                return d.strftime("%H:%M")
    return np.nan

cols = ['FromAirport_Time', 'ToAirport_Time']
for col in cols:
    routes[col] = routes[col].apply(traitement_time)

In [78]:
print(routes.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 789980 entries, 0 to 789979
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   CallSign          789979 non-null  object
 1   FromAirport_IATA  789980 non-null  object
 2   FromAirport_Time  1842 non-null    object
 3   ToAirport_IATA    789980 non-null  object
 4   ToAirport_Time    1735 non-null    object
 5   RouteStop         789980 non-null  bool  
 6   Operator_IATA     258249 non-null  object
dtypes: bool(1), object(6)
memory usage: 36.9+ MB
None


In [79]:
routes.columns = ['callsign', 'airport_from_iata', 'airport_from_time', 'airport_to_iata', 'airport_to_time', 'hasStopover', 'airline_iata']
routes = routes[['callsign', 'airline_iata', 'airport_from_iata', 'airport_from_time', 'airport_to_iata', 'airport_to_time', 'hasStopover']]
routes.columns = ['callsign', 'fk_airline_iata', 'fk_airport_from_iata', 'airport_from_time', 'fk_airport_to_iata', 'airport_to_time', 'hasStopover']
routes['hasStopover'] = routes['hasStopover'].apply(lambda x: 1 if x == True else 0)

In [80]:
routes = routes.dropna(subset=['callsign'])
# pb avec le callsign FR3652 qui apparait 2 fois en sql
routes = routes[routes['callsign'] != 'FR3652']
routes = routes.reset_index(drop=True)

In [81]:
print(df_escales.columns)
print(df_escales.dtypes)

Index(['escale_route_callsign', 'escale_number_total', 'escale_order_number',
       'airport_iata'],
      dtype='object')
escale_route_callsign    object
escale_number_total       int64
escale_order_number       int64
airport_iata             object
dtype: object


In [82]:
df_escales.columns = ['fk_route_callsign', 'escale_total_number', 'escale_order_number', 'fk_airport_iata']
df_escales = df_escales[['fk_route_callsign', 'fk_airport_iata', 'escale_total_number', 'escale_order_number']]

In [83]:
duplicate = df_escales.duplicated(subset=['fk_route_callsign', 'fk_airport_iata'], keep=False)
df_duplicate = df_escales[duplicate]
callsigns_duplicated = df_duplicate['fk_route_callsign'].unique().tolist()
df_escales = df_escales.drop_duplicates(subset=['fk_route_callsign', 'fk_airport_iata'], keep=False)
df_escales = df_escales.reset_index(drop=True)
routes.loc[routes['hasStopover'].isin(callsigns_duplicated), 'hasStopover'] = 0
df_escales = df_escales.astype({
    'escale_order_number': int
})
df_escales

,fk_route_callsign,fk_airport_iata,escale_total_number,escale_order_number
0,NVR828,DXB,1,1
1,GLG38,GYE,1,1
2,OAW349,BDS,1,1
3,OAW355,PMO,1,1
4,OAW367,CTA,1,1
...,...,...,...,...
25722,JAF12A,CHQ,1,1
25723,FCB241,ATH,1,1
25724,UAE9374,BRU,2,1
25725,UAE9374,LCK,2,2


In [84]:
# on exporte les df en csv
# on utilisera un fichier sql pour remplir les tables de la db
os.makedirs(path_folder_output_csv, exist_ok=True)
airport.to_csv(f"{path_folder_output_csv}/airports.csv", index=False)
aircraft.to_csv(f"{path_folder_output_csv}/aircrafts.csv", index=False)
airline.to_csv(f"{path_folder_output_csv}/airlines.csv", index=False)
country.to_csv(f"{path_folder_output_csv}/countries.csv", index=False)
city.to_csv(f"{path_folder_output_csv}/cities.csv", index=False)
routes.to_csv(f"{path_folder_output_csv}/routes.csv", index=False)
df_escales.to_csv(f"{path_folder_output_csv}/stopovers.csv", index=False)

routes.to_csv(f"{path_folder_output_csv}/routes.csv", index=False)
df_escales.to_csv(f"{path_folder_output_csv}/stopovers.csv", index=False)